In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/agribrain/deployment-workflow/dep/

/content/drive/MyDrive/agribrain/deployment-workflow/dep


In [ ]:
!ls

agscripts  api-design.ipynb   kmodels		     txt-clusters
api	   cluster-summaries  knowledge-graph.ipynb


In [ ]:
!pip install -q transformers
!pip install -q torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.5 MB/s eta 0:00:00


In [ ]:
import os
from tqdm import tqdm
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
model_name = "philschmid/bart-large-cnn-samsum"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [ ]:
def summarize_text(large_text):
  inputs = tokenizer(large_text, max_length=1024, truncation=True, return_tensors="pt")

  summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=2048, early_stopping=True)
  summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
  return summary

In [ ]:
!ls

agscripts  api-design.ipynb   kmodels		     txt-clusters
api	   cluster-summaries  knowledge-graph.ipynb


In [ ]:
def summarize_clusters(directory):
    text_data = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as f:
                text_data.append(f.read())
    content = "\n\n".join(text_data)
    summary_dir = os.path.join(directory, "summary/")
    if not os.path.exists(summary_dir):
      os.mkdir(summary_dir)
    summary_file = os.path.join(summary_dir, "summary.txt")
    with open(summary_file, mode="w", encoding="utf-8") as fp:
      summary = summarize_text(content)
      fp.write(summary)

In [ ]:
def get_cluster_directories(path):
    directories = []
    for entry in os.scandir(path):
        if entry.is_dir():
            directories.append(entry.path)
    return directories

In [ ]:
cluster_dirs = get_cluster_directories("./txt-clusters/")

for cluster_dir in tqdm(cluster_dirs):
  summarize_clusters(cluster_dir)

100%|██████████| 128/128 [1:08:59<00:00, 32.34s/it]
